In [1]:
# import torch
import os
import json
import pandas as pd

`ajId`:ID of the case,   
`ajName` is the case name,   
`ajjbqk` 基本事实,  
`pjjg` 判决,    
`qw`  全部内容,   
`writId` 文档ID,   
`writName` 文档名称   

In [3]:
# 预处理数据
data = []
for directory, dirnames, filenames in os.walk(r'LeCaRD/data/candidates'):
	for name in filenames:
		json_file = os.path.join(directory, name)
		data.append(json.load(open(json_file,encoding="utf-8")))
data = pd.DataFrame(data)
data = data[["qw"]].to_dict()["qw"]
data = [data[i] for i in range(len(data.keys()))]
documents = data # 换个名字

# 读取停用词
with open("LeCaRD/data/others/stopword.txt",encoding="utf-8") as f:
	stopwords = set(f.read().split("\n"))
stopwords.add(" ")

# 构建倒排索引
1. 分词和去除停用词


In [31]:
import jieba
import tqdm
from collections import defaultdict
import pickle
import numpy as np
import sqlite3
from scipy import sparse
from scipy.sparse.linalg import norm
from typing import List

In [33]:
conn = sqlite3.connect("index.db")
cur = conn.cursor()
matrix = sparse.load_npz("./document_matrix.npz")
matrix = matrix.T
cur.execute("""select * from words""")
words = [word[1] for word in cur.fetchall()]
cur.execute("""select * from dc""")
word_doc_count = [dc[1] for dc in cur.fetchall()]
words_idx = {word:idx for idx,word in enumerate(words)}


In [212]:
def get_documents_matrix(keys:list):
	l = []
	for key in keys:
		if key in words:
			l.append(words_idx[key])
	sql = """select document from inverse_index where word_index in ({})""".format(",".join(["?"]*len(l)))
	cur.execute(sql,l)
	return np.array(list({i[0] for i in cur.fetchall()}))
	
	
# def get_document(word):
# 	if word in words:
# 		word_index = words.index(word)
# 	else:
# 		return []
# 	cur.execute("""select document from inverse_index where word_index=?""",(word_index,))
# 	return [i[0] for i in cur.fetchall()]


def query(key:str,n:int=20):
	key_with_count = pd.Series(list(jieba.cut(key))).value_counts().to_dict()
	_x = sparse.lil_matrix((len(words),1))
	# 生成查询向量的tf-idf
	for word in key_with_count:
		if word in words:
			pos = words_idx[word]
			_x[pos,0] = key_with_count[word] * np.log( (len(documents)+1) / (1 + word_doc_count[pos]))
	x = _x.tocsr()
	# 分词，利用倒排索引
	docuement_to_query = get_documents(key_with_count.keys())
	m = matrix[docuement_to_query]
	# 计算查询向量与文档向量的余弦相似度
	cosine_distance = (m@x).toarray().flatten()/(norm(m,axis=1) * norm(x))
	ind = np.argpartition(cosine_distance, -n)[-n:]
	return docuement_to_query[ind[np.argsort(cosine_distance[ind])]]

def get_document(document_id:int):
	return documents[document_id]

class DocumentKeywordPos:
	def __init__(self,start:int,end:int):
		self.start = start
		self.end = end
	def __repr__(self):
		return f"{self.start}:{self.end}"
def get_document_with_hightlight(document_id,keys:List[str])->Tuple[str,List[DocumentKeywordPos]]:
	re_expr = "|".join(["({})".format(re.escape(key)) for key in keys])
	document = get_document(document_id)
	hightlight = [DocumentKeywordPos(match.start(),match.end()) for match in re.finditer(re_expr, document)]
	return document,hightlight